In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import SAC

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "SAC3"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [4]:
folder = '/mnt/d/donkeycar_model/SAC3/'

In [5]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [6]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
loading scene mini_monaco


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 7.13 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 8.49 seconds


In [8]:
for i in range(1000):
    # preparation
    if i == 0:
        model = model = SAC("CnnPolicy", env, n_steps=500, verbose=0, buffer_size = 10000, learning_rate = 0.00000005 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_SAC3-{(i+1)*2500}.npy", Log)

/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


testing 2500 : 63.69296636640072
testing 5000 : 62.4270812958908
testing 7500 : 61.804798566308804
testing 10000 : 62.663177435329644
testing 12500 : 65.931326672842
testing 15000 : 68.42376494273398
testing 17500 : 76.79919890865108
testing 20000 : 126.66712746240125
testing 22500 : 135.53556983261348
testing 25000 : 143.69886722091744
testing 27500 : 149.72861929904545
testing 30000 : 149.9586879119459
testing 32500 : 149.21540183042788
testing 35000 : 147.0008694417652
testing 37500 : 146.58888489969246
testing 40000 : 141.3212293386543
testing 42500 : 138.71665452706281
testing 45000 : 139.62409165358787
testing 47500 : 142.08420899834363
testing 50000 : 142.8423051876072
testing 52500 : 142.74246475313856
testing 55000 : 144.84874856088027
testing 57500 : 145.87262397211063
testing 60000 : 142.84819117226363
testing 62500 : 141.4563429912502
testing 65000 : 140.8768728734496
testing 67500 : 140.70805684512592
testing 70000 : 141.37803986103125
testing 72500 : 138.3198734660645
tes

KeyboardInterrupt: 

In [9]:
env.close()